In [8]:
import tensorflow as tf

In [6]:
class switch(object):
    """Switch statement for Python, based on recipe from Python Cookbook."""

    def __init__(self, value):
        self.value = value
        self.fall = False

    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration
    
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args: # changed for v1.5
            self.fall = True
            return True
        else:
            return False

In [11]:
# test_dict = {"hi": 5, "hey": 3}
# test_dict = ["hi", "hey"]
test_dict = "hi"

for case in switch(test_dict):
    print(case("hi"))
    if case('dfsd'):
        print(55)
    elif case('hi'):
        print(23423)

True
55


RuntimeError: generator raised StopIteration

In [9]:
# https://stats.stackexchange.com/questions/330176/what-is-the-output-of-a-tf-nn-dynamic-rnn
tf.reset_default_graph()
n_steps = 2
n_inputs = 3
n_neurons = 5

X = tf.placeholder(dtype=tf.float32, shape=[None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, sequence_length=seq_length, dtype=tf.float32)

X_batch = np.array([
  # t = 0      t = 1
  [[0, 1, 2], [9, 8, 7]], # instance 0
  [[3, 4, 5], [0, 0, 0]], # instance 1
  [[6, 7, 8], [6, 5, 4]], # instance 2
  [[9, 0, 1], [3, 2, 1]], # instance 3
])
seq_length_batch = np.array([2, 1, 2, 2])


In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_val, states_val = sess.run([outputs, states],
                                       feed_dict={
                                           X: X_batch,
                                           seq_length: seq_length_batch
                                       })

    print(outputs_val.shape)
    print()
    print(states_val.shape)

(4, 2, 5)

(4, 5)


In [19]:
tf.reset_default_graph()
# x = tf.constant([[1., 1.], [2., 2.]])
x = tf.random.uniform((4,5,6,7))
y = tf.reduce_mean(x,axis=1)
z = tf.transpose(x, perm=[0,1,3,2])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(x.shape)
    print(y.shape)
    print(z.shape)
# x.shape

ValueError: Dimension must be 4 but is 3 for 'transpose' (op: 'Transpose') with input shapes: [4,5,6,7], [3].